## Mutliple Inheritance and method-resolution order

```python
class SubClass(BaseClass):
    pass
```

* SubClass inherits all attributes and may override methods of BaseClass
* it is generally necessary to call base class initializers to ensure proper object initialization
* If a subclass doesn't define an initializer, then the base class initializer is called during construction

In [4]:
class Base:
    def __init__(self):
        print('Base initializer')
        
    def f(self):
        print('Base.f()')
        
        
# Sub doesn't define its initializer, so Base' initializer will be called
# Sub also inherit f()
class Sub(Base):
    pass

In [2]:
b= Base()

Base initializer


In [3]:
b.f()

Base.f()


In [5]:
c = Sub()

Base initializer


In [6]:
c.f()

Base.f()


In [7]:
class Base:
    def __init__(self):
        print('Base initializer')
        
    def f(self):
        print('Base.f()')
        
        
# Sub doesn't define its initializer, so Base' initializer will be called
# Sub also inherit f()
class Sub(Base):
    
    # override f
    def f(self):
        print('Sub.f()')

In [8]:
c = Sub()

Base initializer


In [9]:
c.f()

Sub.f()


#### Sub-class initializer
* if sub class initializer does not automatically call the base class initializers
* base class initializer is executed if sub class initializer explicitly calls it, or sub class doesn't define initializer

#### sub-class initializer dose not explicitly call the base call initializer

In [12]:
class Base:
    def __init__(self):
        print('Base initializer')
        
    def f(self):
        print('Base.f()')
        
        
# Sub doesn't define its initializer, so Base' initializer will be called
# Sub also inherit f()
class Sub(Base):
    
    def __init__(self):
        print('Sub initializer')
    
    # override f
    def f(self):
        print('Sub.f()')

In [13]:
c= Sub()

Sub initializer


#### sub-class initializer explicitly call the base call initializer

In [14]:
class Base:
    def __init__(self):
        print('Base initializer')
        
    def f(self):
        print('Base.f()')
        
        
# Sub doesn't define its initializer, so Base' initializer will be called
# Sub also inherit f()
class Sub(Base):
    
    def __init__(self):
        super().__init__()
        print('Sub initializer')
    
    # override f
    def f(self):
        print('Sub.f()')

In [15]:
c= Sub()

Base initializer
Sub initializer


#### Using SimpleList class to explore inheritance in Python

In [33]:
class SimpleList:
    def __init__(self, items):
        self._items = list(items)
        
    def add(self, item):
        self._items.append(item)
        
    def __get_item__(self, index):
        return self._items[index]
    
    def sort(self):
        self._items.sort()
        
    def __len__(self):
        return len(self._items)
    
    def __repr__(self):
        return f"{type(self).__name__}({self._items!r})"
    
class SortedList(SimpleList):
    def __init__(self, items=()):
        super().__init__(items)
        self.sort()
        
    def add(self, item):
        super().add(item)
        self.sort()
    

### Type inspection functions
#### isinstance()
* isinstance returns True if the first argument is an instance of the second argument, which is a class
  + it also returns True if the first argument is a subclass of the second argument
* isinstance also accept its second argument as a tuple of types, and returns True if the first argument is the subclass or class of any of the second argument  

In [34]:
s1 = SortedList([3,2,1])

In [35]:
print(isinstance(s1, SortedList))

True


In [36]:
print(isinstance(s1, SimpleList))

True


In [37]:
# returns True because x is an instance of list
x =[]
isinstance(x, (float, dict, list))

True

In [38]:
class IntList(SimpleList):
    def __init__(self, items=()):
        for x in items: self._validate(x)
        super().__init__(items)
        
    @staticmethod
    def _validate(x):
        if not isinstance(x, int):
            raise TypeError('IntList only supports integer values')
            
    def add(self, item):
        self._validate(item)
        super().add(item)

In [39]:
il = IntList([1, 2, 3, 4])
il.add(19)

In [40]:
il.add(3.4)

TypeError: IntList only supports integer values

#### issubclass
* operates on types to check for sub/superclass relationships
* determines if one class is a subclass of another
* takes two arguments, both must be types
* returns True if the first argument is a direct or indirect subclass of the second

In [41]:
print(issubclass(IntList, SimpleList))
print(issubclass(SortedList, SimpleList))
print(issubclass(SortedList, IntList))

True
True
False


In [42]:
# show indirect relationship
class MyInt(int): pass
class MyVerySpecialInt(MyInt): pass
issubclass(MyVerySpecialInt, int)

True

### Multiple Inheritance
* defining a class with more than one direct base class
* not universal among object-oriented languages
* can lead to certain complexities
  + when multiple base classes define the same method
  + python has a relatively simple system for dealing with them

```python
class SubClass(Base1, Base2, Base3):
    pass
```

#### Name Resolution with multiple base classes
* classes inherit all methods from all of their bases
* if there is no meethod name overlap, names resolve to the obvious method
* in the case of overlap, python uses a well-defined method resolution order to decide which to use
* if a class uses multiple inheritance and defines no initializer, only the initializer of the first base class is automatically called
* \_\_bases\_\_ returns a list of all the base classes
* \_\_base\_\_ returns the first base classes

In [43]:
class SortedIntList(IntList, SortedList):
    pass

In [49]:
sil = SortedIntList([42, 23, 3])
sil

SortedIntList([3, 23, 42])

In [50]:
sil.add(-100)
sil

SortedIntList([-100, 3, 23, 42])

In [51]:
sil.add(3.4)

TypeError: IntList only supports integer values

In [53]:
SortedIntList.__bases__

(__main__.IntList, __main__.SortedList)

#### Method resolution order
* ordering of an inheritance graph that determins which implementation to use when invoking a method
* The Method Resolution Order for a class is stored on a special member called \_\_mro\_\_
  + C3 algoritm is used to calculate the MRO of a class. The principle is
    + subclasses come before base classes
    + base class order from class definition is preserved
    + all MROs in a proram fllow the above two rules 
  + C3 algorithm prohibits some inheritance declarations in Python
    + some base classes declarations will violate C3 and python will refuse to compile them, as shown below:
    since both B and C inherit A, for class D, it is impossible to caclculate mro for D, because A should be placed after C (C inherits A), but the base class list of D requires A to be placed before C
    ```python
    class A: pass
    class B(A): pass
    class C(A): pass
    class D(B, A, C): pass
    ```
* How python use MRO?
  + finds the MRO of the type of the object on which a method is invoked
  + checks each class in the MRO and find the first one that implements the method and stops

In [54]:
IntList.__mro__

(__main__.IntList, __main__.SimpleList, object)

In [61]:
SortedIntList.__mro__

(__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object)

In [57]:
# example of classes in a diamond inheritance graph

class A:
    def func(self):
        return 'A.func'
    
class B(A):
    def func(self):
        return 'B.func'

class C(A):
    def func(self):
        return 'C.func'

class D(B, C):
    pass

In [58]:
# the MRO for d is: itself, B, C, A and object
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

In [60]:
# B is the first class in MRO that defines the function func
d = D()
d.func()

'B.func'

#### An example of MRO
Question: how is IntList.add() delegating to SortedList.add() in SortedIntList class.
  * since IntList is before SortedList in mro, IntList.add() should be invoked, not the SortedList.add(), but when adding new elements, SortedList.add() will be invoked
Answer: 
  * this is due to how super() works.
    + super() gives you a proxy object, which resolves the correct implementation by searching requested mehtod from the MRO
    + super() works with the MRO of the object originally invoking the method, not just its base class
    + given a MRO and a class c in that MRO, super() gives an object which resolves method using only the part of teh MRO coming after c
  * when SortedInList calls add(), it wil call IntList.add(), which then calls its super().add().
  * super() uses two argurments
    + one is the invoking class, which is IntList
    + the other is the self object of the invoking method. as shwon below: 
    ```python
    def add(self, item):
        self._validate(item)
        super().add(item)
    ```    
    + the self object in this IntList.add() is a SortedIntList object, whose MRO is:
      + (itself, IntList, SortedList, SimpleList, object)
    + use the elements after the invoking class, which is IntList, so the mro that super() will look for is
      + (SortedList, SimpleList, object)
    + the add method of SortedList will be used. This also applies to \_\_init\_\_() method
  * this is surprising, since SortedList is not listed anywhere in IntList's inheritance graph, and its base class is SimpleList
  * this is because super() uses the full MRO of the self object's class, not just the base classes from a class definition of the method invoking class
    + this is also call that super() is instance bound, since it is bounded by the self object of the method calling super()
Code demo:
  * when SortedIntList() is called, it calls the \_\_init\_\_() of IntList, which provides two arguments:
    + IntList class, which is the class where the super() is called     
    + a SortedIntList object , which is the self object of \_\_init\_\_(self, items=()), here is a SortedIntList object

In [69]:
class IntList(SimpleList):
    def __init__(self, items=()):
        for x in items: self._validate(x)
        s = super()
        print(s)
        s.__init__(items)
        
    @staticmethod
    def _validate(x):
        if not isinstance(x, int):
            raise TypeError('IntList only supports integer values')
            
    def add(self, item):
        self._validate(item)
        super().add(item)
        
class SortedIntList(IntList, SortedList):
    pass        
        

In [70]:
sil = SortedIntList()

<super: <class 'IntList'>, <SortedIntList object>>


#### calling super() in a classmethod
* in this case, we don't have an instance to work with, but we do have a class object
* super() derives the MRO from the class object rather than the type of self    

In [71]:
class Animal:
    @classmethod
    def description(cls):
        return "An animal"
    

class Bird(Animal):
    @classmethod
    def description(cls):
        s = super()
        print(s)
        return s.description() + " with wings"
    
class Flamingo(Bird):
    @classmethod
    def description(cls):
        return super().description() + " and fabulous pink feathers"

In [72]:
# as demonstrated following, 
# the first arg is the class Bird, where the super() is invoked
# the second is the cls transferred to Bird.description, which is an Flamingo object
Flamingo.description()

<super: <class 'Bird'>, <Flamingo object>>


'An animal with wings and fabulous pink feathers'

#### Explicit arguments to super()
* we can call super explicitly by super(class-object, instance-or class)
  + the first is used to trim mro
  + the second is to provide the mro

In [73]:
s = SortedIntList()
super(IntList, s).add

<super: <class 'IntList'>, <SortedIntList object>>


<bound method SortedList.add of SortedIntList([])>

In [74]:
# provide the class and instance for super() proxy, and then invoke the add method
# this bypass the super() method in Inlist for int checking.
super(IntList, s).add("I am not a number, I am a free man!")
s

SortedIntList(['I am not a number, I am a free man!'])

### Duck type
* in python, the type of an object doesn't determine if it can be used in a particular context
* python uses duck typing where fitness for purpose is dtermined at the time of use. if a required attribute is not avalialbe, an exception will be thrown
* functions don't specify their types
* you can call any method on any object, and python won't complain until runtime
* inheritance in Python is a convenient way to reuse code, much more than itis a way to construct type hieararchies
* 